# Q-Learning algorithm for cart-pole

The following is an exercise in reinforcement learning. We consider a basic problem from the `OpenAI-Gym`. In which we use a Q-Learner algorithm to have a neural network converge to a state in which it can balance a stick on a moveable cart.

At any given point in time the state of the system is defined by its position and the angle of the stick. If the angle of the stick is too big, the stick falls. To keep that from happening an action can be performed, that is the Q-Learner can move the cart left or right with a given velocity. (See below for a graphical representation of this)

**Q-Learning** means that the input of the neural network is a state combined with a possible action to take in that state. (That is the position of the cartpole combined with the action to move it left or right -- we neglect its velocity for simplicity). The output of the neural network is an value estimating how good that action is (Q-value).

**We train the neural network by** using its own training episodes in which we approximate the Q-value by taking it dependent on how long it manages to maintain the stick balanced (see below).

Once the algorithm is trained, one can select an action by selecting the action with the highest Q-value or by treating them as probabilities to introduce some randomness.

We will use the `tensorflow.Keras`-framework.

In [ ]:
import gym
import numpy as np
import random
from tensorflow.keras import Model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from _collections import deque

The environment can be set up by running:

In [ ]:
env = gym.make('CartPole-v1')

Let us fix some parameters before we begin. The two most interesting ones are:
1. `relative_randomness`
2. `randomness_decay`
The reason behind these parameters is that an initial state of a neural network might include some initial bias towards certain actions that keep it from generating unbiased testing date.
So we start out by selecting random moves, over time the neural network starts converging towards a state in which its actions are better, such that we take less and less random moves.

In [ ]:
batch_size = 32
buffer_capacity = 1000
episodes_until_abortion = 10000
relative_randomness = 1
randomness_decay = 0.95

Let us define a utility class which holds an *experience buffer* with a maximum capacity specified by 'capacity' to which we append training episodes. We can then retrieve randomized batches of episodes:

In [ ]:
class ExperienceBuffer:
    def __init__(self, capacity, batch_size):
        self.buffer = deque()
        self.capacity = capacity
        self.batch_size = batch_size

    def save_step(self, state, action, reward, next_state):
        self.buffer.append((state, action, reward, next_state))
        if len(self.buffer) > self.capacity:
            self.buffer.popleft()

    def get_experience_batch(self):
        return random.sample(self.buffer, min(len(self.buffer), self.batch_size))

We initialize it using the parameters specified above:

In [ ]:
xp_buffer = ExperienceBuffer(batch_size=batch_size, capacity=buffer_capacity)

## The Q-Learning algorithm:
We encapsulate the Q-Learning algorithm in a class:
* The constructor contains the topology of the neural network and some hyperparameters:
    1. We use three linear layers with a `relu`-activation layer the last of which has the output-shape of just one number: The Q-value.
    2. The learning rate decays exponentially
    3. We use the `Adam`-optimizer with mean squared errors as the loss function.
    4. Finally, we specify a parameter `gamma`, which controls at what step number we expect the outcome to be correlated with an action (i.e. a positive/negative outcome should not affect our estimate for the reward of an action taken many steps before)
* The `preprocess` function turns a list of experience tuples `(state, action, reward, next_state)` into inputs and outputs for the neural net.
* the `get_action` function returns an action based upon an input state either using the neural net or randomly (if the randomness `eps` is non-zero)

In [ ]:
class QLearner:
    worst_reward = -10
    all_actions = np.array([0, 1])
    gamma = 0.75
    dense_layer_size = 24
    normalization = [0.42, 1]
    
    def __init__(self, weight_file):
        self.weight_file = weight_file
        self.glue = lambda states, actions: np.concatenate((states[:, 2:] / self.normalization,
                                                            actions[:, None]), axis=1)

        initializer = RandomNormal(stddev=0.1)
        input_layer = Input(shape=(3,), dtype='float32')
        layers = [input_layer]
        layers.append(Dense(units=self.dense_layer_size, activation='relu', kernel_initializer=initializer)(layers[-1]))
        layers.append(Dense(units=self.dense_layer_size, activation='relu', kernel_initializer=initializer)(layers[-1]))

        output_layer = Dense(units=1, activation='relu')(layers[-1])

        learning_rate = ExponentialDecay(initial_learning_rate=1, decay_rate=0.96, decay_steps=1)
        optimizer = Adam(learning_rate=learning_rate)
        self.model = Model(input_layer, output_layer)
        self.model.compile(optimizer=optimizer, loss='mse')
        self.load_weights()

    def load_weights(self):
        try:
            self.model.load_weights(self.weight_file)
            print("Weights loaded from file.")
        except:
            print("No weight file found.")
            pass

    def save_weights(self):
        self.model.save_weights(self.weight_file)

    def preprocess(self, experience):
        states = np.array([xp[0] for xp in experience])
        actions = np.array([xp[1] for xp in experience])
        rewards = np.array([xp[2] for xp in experience])
        next_states = np.array([xp[3] for xp in experience])

        neural_net_inputs = self.glue(states, actions)

        q_values_go_left = self.model.predict(self.glue(next_states, np.zeros(len(next_states))))[:, 0]
        q_values_go_right = self.model.predict(self.glue(next_states, np.ones(len(next_states))))[:, 0]

        # this is the actual Q-learning policy!
        neural_net_outputs = rewards + self.gamma * np.maximum(q_values_go_left, q_values_go_right)
        
        # This is when it the stick fell down
        neural_net_outputs[-1] = self.worst_reward

        return neural_net_inputs, neural_net_outputs

    def get_action(self, state, eps=0):
        take_rand_choice = np.random.choice(self.all_actions, p=[1.0 - eps, eps]) != 0
        if take_rand_choice:
            return np.random.choice(self.all_actions)
        else:
            inp = self.glue(np.array([state, state]), np.array([0, 1]))
            pred = self.model.predict(inp)[:, 0]
            return self.all_actions[np.where(pred == max(pred))][0]

    def train(self, inputs, outputs, verbose=1):
        self.model.evaluate(inputs, outputs, batch_size=len(inputs), verbose=verbose)

## A look at the problem before we train
Before we start to train the neural network, let us look at the problem and see how the untrained agent performs.

We render multiple episodes, because they are very short prior to training.

(A window should open)

In [ ]:
agent = QLearner(weight_file="cart_pole")

def render_episode(agent):
    step_no = 0
    state = env.reset()
    stick_fallen_down = False
    
    while not stick_fallen_down:
        env.render()
        action = agent.get_action(state)
        state, reward, stick_fallen_down, info = env.step(action)
        step_no += 1
        
    env.close()
        
    print("Episode lasted", step_no, "steps.")

for i in range(10):
    render_episode(agent)

Before we finally start the training loop, we define some helpful functions:
* A function to perform one training step and save it
* The randomness decay. We reset back to 100% moves if the QLearner fails to start 
learning.
* A function getting the next action based upon the current state from the QLearner and saves it to the buffer.

In [ ]:
max_steps_to_reset = 20

def train():
    experience = xp_buffer.get_experience_batch()
    inputs, outputs = agent.preprocess(experience)
    agent.train(inputs=inputs, outputs=outputs, verbose=0)
    
def get_new_relative_randomness(relative_randomness, avg_steps):
    if avg_steps < max_steps_to_reset:
        return 1
    else:
        return round(relative_randomness * randomness_decay, 5)
    
def reset_agent_if_necessary(agent, avg_steps):
    if avg_steps < max_steps_to_reset:
        agent = QLearner(weight_file="cart_pole")
        
def perform_action_and_save(state):
    action = agent.get_action(state, eps=relative_randomness)
    next_state, reward, stick_fallen_down, info = env.step(action)
    xp_buffer.save_step(state, action, reward, next_state)
    return (stick_fallen_down, next_state)

def simulate_episode():
    state = env.reset()
    stick_fallen_down = False
    step_no = 0
    while not stick_fallen_down:
        (stick_fallen_down, state) = perform_action_and_save(state)
        step_no += 1
    return step_no

## The training loop
Finally we are ready, to train. We initialized the QLearner and proceed as follows.
1. We loop over as many training episodes as allowed above
2. In every loop we balance the stick for as long as possible using a combination of random moves and moves predicted by the QLearner. Then we save them to the buffer.
3. Once we have some episodes in the buffer we start training the neural network on every cycle
4. Every `100` episodes, we decrease the randomness of the actions used
5. If the average number of steps every falls below a certain value, we empirically found that it's hard for the algorithm to improve, such that we reset and start again.
6. If the average step number exceeds `200`, we consider the problem as solved.

In [ ]:
checkpoint_after_n_episodes = 100
steps_for_problem_to_be_solved = 200
episodes_to_average_over = 100
minimum_buffer_size = 5
status_fmt_str = "{}: avg. steps in the last {} episodes: {}. relative share of random moves = {}"

step_no = 0
for i in range(episodes_until_abortion):
    step_no += simulate_episode()

    if i > minimum_buffer_size:
        train()

    if i % checkpoint_after_n_episodes == 0 and i > 0:
        avg_steps = step_no/episodes_to_average_over
        print(status_fmt_str.format(i, episodes_to_average_over, avg_steps, relative_randomness))
        relative_randomness = get_new_relative_randomness(relative_randomness, avg_steps)
        reset_agent_if_necessary(agent, avg_steps)
        step_no = 0

    if step_no/episodes_to_average_over >= steps_for_problem_to_be_solved:
        print("Problem is solved!")
        agent.save_weights()
        break

env.close()

Let us once again render a couple of episodes of the agent to get an idea how it performs now:

(You can load the weights for the model I trained by uncommenting the line)

In [ ]:
#agent = QLearner(weight_file="cart_pole_demo")
for i in range(5):
    render_episode(agent)

## Possible improvements:
* To keep the model simple, we have completely disregarded the velocity component of the cart, including into a more complex model is likely to further improve the performance
* We used a simple exponential decay for the randomness. This had the drawback that we needed to set a very slow decay to be sure that the learning started at the start, while it could be faster later on. This could be either implementing by having an accelerated decay or by implementing an adaptive decay